# Descripción del problema


Mi padre solía participar frecuentemente en las dinámicas que organizaba la cadena de supermercados "Super del Norte" para sus clientes. Entre las dinámicas había rifas, regalos aleatorios y la más relevante para este proyecto: un juego en el que un cliente seleccionado debía recorrer rápidamente los pasillos del súper con un carrito de compra tomando todo lo que quisiera hasta que se llenara su carrito o se le acabara el tiempo de la dinámica. 

Esto presenta una idea interesante si se plantea como un problema de optimización: el carrito de compra se puede llenar con muchas combinaciones diferentes de productos, pero solo algunas combinaciones (o solo una) tendrán un valor máximo según el precio de los artículos contenidos y las cantidades de dichos artículos. Por simplicidad, olvidemos el límite de tiempo que tienen los participantes para llenar su carrito e imaginemos que pueden ir por los artículos que deseen con calma y sin prisa. También supongamos que los productos se acomodan perfectamente para llenar el espacio del carrito, y que no queda espacio vacío entre ellos.

# Datos
Usaremos los datos de la encuesta "¿Quién es Quién en los Precios?" de PROFECO, ya que éstos datos contienen información sobre diferentes productos en venta en las diferentes tiendas, como por ejemplo el nombre del producto, la presentación, la marca, categoría, el precio, la fecha de registro, la cadena, el giro, dirección, coordenadas, estado y municipio. Esto nos servirá para modelar el problema.

In [39]:
import pandas as pd
import os
import unicodedata
import re
import numpy as np

En particular, usaremos un archivo de datos pertenecientes al 2025, segundo archivo del décimo periodo (archivo `10-2025_02.csv`). Se descarga, descomprime y se pone en la carpeta `/data`:

In [9]:
dataset = "../data/10-2025_02.csv"
df_10_25_02 = pd.read_csv(dataset, encoding="latin1")
print(df_10_25_02.shape)
df_10_25_02.head(3)

(595062, 15)


producto                presentacion marca           categoria  \
0    ACELGA                      MANOJO   S/M  HORTALIZAS FRESCAS   
1  AGUACATE          1 KG. GRANEL. HASS   S/M      FRUTAS FRESCAS   
2       AJO  1 KG. GRANEL. O AJO MORADO   S/M  HORTALIZAS FRESCAS   

             catalogo  precio fecha_registro    cadena_comercial  \
0  FRUTAS Y LEGUMBRES    15.0     16/10/2025  CENTRAL DE ABASTOS   
1  FRUTAS Y LEGUMBRES    39.0     16/10/2025  CENTRAL DE ABASTOS   
2  FRUTAS Y LEGUMBRES   120.0     16/10/2025  CENTRAL DE ABASTOS   

                giro   nombre_comercial  \
0  CENTRAL DE ABASTO  CENTRAL DE ABASTO   
1  CENTRAL DE ABASTO  CENTRAL DE ABASTO   
2  CENTRAL DE ABASTO  CENTRAL DE ABASTO   

                                           direccion          estado  \
0  AV. MAHATMA GANDHI S/N. SALIDA A MXICO. COL. ...  AGUASCALIENTES   
1  AV. MAHATMA GANDHI S/N. SALIDA A MXICO. COL. ...  AGUASCALIENTES   
2  AV. MAHATMA GANDHI S/N. SALIDA A MXICO. COL. ...  AGUASCALIENTES   

        municipio    latitud    longitud  
0  AGUASCALIENTES  21.832072 -102.292976  
1  AGUASCALIENTES  21.832072 -102.292976  
2  AGUASCALIENTES  21.832072 -102.292976

Se tienen casi 600 mil filas en el archivo completo. Vamos a fijarnos en los posibles valores para 'estado':

In [7]:
df_10_25_02['estado'].unique()

array(['AGUASCALIENTES', 'BAJA CALIFORNIA', 'BAJA CALIFORNIA SUR',
       'CAMPECHE', 'CHIAPAS', 'CHIHUAHUA', 'CIUDAD DE MEXICO', 'COAHUILA',
       'DURANGO', 'ESTADO DE MEXICO', 'GUANAJUATO', 'GUERRERO', 'HIDALGO',
       'JALISCO', 'MICHOACAN', 'MORELOS', 'NUEVO LEON', 'OAXACA',
       'PUEBLA', 'QUERETARO', 'QUINTANA ROO', 'SAN LUIS POTOSI',
       'SINALOA', 'SONORA', 'TABASCO', 'TAMAULIPAS', 'TLAXCALA',
       'VERACRUZ', 'YUCATAN', 'ZACATECAS'], dtype=object)

Nos quedamos con los datos de Sonora:

In [8]:
df_sonora = df_10_25_02[df_10_25_02['estado']=='SONORA']
df_sonora.shape

(7022, 15)

y los datos se reducen a 7022 filas. Ahora observamos los posibles municipios:

In [11]:
df_sonora['municipio'].unique()

array(['HERMOSILLO'], dtype=object)

Vemos que todos los datos pertenecen a Hermosillo.

In [13]:
df_sonora['cadena_comercial'].unique()

array(['TORTILLERIAS TRADICIONALES', 'HIPERMERCADO SORIANA', 'LEY',
       'MERCADO PUBLICO', 'ELEKTRA', 'BENAVIDES', 'FARMACIA GUADALAJARA',
       'WAL-MART', 'BODEGA AURRERA', 'COPPEL', 'SEARS ROEBUCK DE MEXICO',
       'FARMACIAS DEL AHORRO'], dtype=object)

In [14]:
df_sonora['giro'].unique()

array(['TORTILLERÖAS', 'SUPERMERCADO / TIENDA DE AUTOSERVICIO',
       'MERCADOS', 'TIENDA DE ELECTRODOM\x90STICOS', 'FARMACIAS',
       'TIENDA DEPARTAMENTALES'], dtype=object)

Por simplicidad vamos a enfocarnos en solamente una cadena, supongamos Walmart:

In [15]:
df_walmart = df_sonora[df_sonora['cadena_comercial']=='WAL-MART']
df_walmart.shape

(1110, 15)

Los datos se reducen a 1110 filas.

In [17]:
df_walmart.head(3)

producto                                   presentacion  marca  \
488919   A.S.COR  FRASCO GOTERO 24 ML. 1.000 G., SOLUCIàN GOTAS    S/M   
488920  ACARBOSA                 CAJA CON 30 TABLETAS DE 50 MG.    S/M   
488921    ACEITE                          BOTELLA 1 LT. VEGETAL  1_2_3   

                                categoria      catalogo  precio  \
488919                       MEDICAMENTOS  MEDICAMENTOS   560.0   
488920                       MEDICAMENTOS  MEDICAMENTOS    83.0   
488921  ACEITES Y GRASAS VEG. COMESTIBLES       BASICOS    38.0   

       fecha_registro cadena_comercial                                   giro  \
488919     21/10/2025         WAL-MART  SUPERMERCADO / TIENDA DE AUTOSERVICIO   
488920     21/10/2025         WAL-MART  SUPERMERCADO / TIENDA DE AUTOSERVICIO   
488921     21/10/2025         WAL-MART  SUPERMERCADO / TIENDA DE AUTOSERVICIO   

                          nombre_comercial  \
488919  WALMART SUCURSAL BOULEVARD MORELOS   
488920  WALMART SUCURSAL BOULEVARD MORELOS   
488921  WALMART SUCURSAL BOULEVARD MORELOS   

                                                direccion  estado   municipio  \
488919  BLVD. JOSE MARIA MORELOS 355. ESQ. CONGRESO. C...  SONORA  HERMOSILLO   
488920  BLVD. JOSE MARIA MORELOS 355. ESQ. CONGRESO. C...  SONORA  HERMOSILLO   
488921  BLVD. JOSE MARIA MORELOS 355. ESQ. CONGRESO. C...  SONORA  HERMOSILLO   

          latitud    longitud  
488919  29.121467 -110.950883  
488920  29.121467 -110.950883  
488921  29.121467 -110.950883

In [20]:
print(len(df_walmart['longitud'].unique()), len(df_walmart['latitud'].unique()))

2 2


Vemos los posibles valores para la latitud:

In [27]:
df_walmart['latitud'].unique()

array([29.121467, 29.066837])

y para la longitud:

In [28]:
df_walmart['longitud'].unique()

array([-110.950883, -110.967247])

Y vemos que solo hay dos ubicaciones. Vamos a quedarnos con una solamente para simular el recorrido de un cliente llenando su carrito, porque solo podría tomar productos contenidos en la sucursal donde se encuentra. 
Del resultado de `df_walmart.head()`, podemos ver que la latitud 29.121467 se empareja con la longitud -110.950883, y que la latitud 29.066837 se empareja con la longitud -110.967247. Estos puntos corresponden al Walmart Morelos y al Walmart del Vado del Río. Yo personalmente prefiero quedarme con el Walmart Morelos porque es el que queda más cerca de mi casa. 

In [31]:
df_walmart = df_walmart[df_walmart['latitud']==29.121467]
df_walmart.shape

(1033, 15)

y finalmente los datos para esa sucursal en particular consisten en 1033 filas.

Como ahora todos los datos son de Sonora, de Hermosillo, y de Walmart, podemos eliminar las columnas de 'estado', 'municipio' y 'cadena_comercial'. Además, tampoco nos sirven para este análisis las columnas de 'marca', 'categoria', 'catalogo', 'fecha_registro', 'giro', 'nombre_comercial', 'direccion', 'latitud' ni 'longitud'. Así que solo nos quedamos con 'producto','presentacion' y 'precio':

In [32]:
df_reducido = df_walmart[['producto','presentacion','precio']]
df_reducido.head(10)

producto                                   presentacion  precio
488919   A.S.COR  FRASCO GOTERO 24 ML. 1.000 G., SOLUCIàN GOTAS   560.0
488920  ACARBOSA                 CAJA CON 30 TABLETAS DE 50 MG.    83.0
488921    ACEITE                          BOTELLA 1 LT. VEGETAL    38.0
488922    ACEITE                        BOTELLA 800 ML. VEGETAL    30.0
488923    ACEITE                         BOTELLA 946 ML. CANOLA    37.0
488924    ACEITE                         BOTELLA 946 ML. CANOLA    37.0
488925    ACEITE                           BOTELLA 946 ML. SOYA    38.0
488926    ACEITE                           BOTELLA 765 ML. MAÖZ    62.0
488927    ACEITE                           BOTELLA 946 ML. SOYA    45.0
488928    ACEITE                        BOTELLA 825 ML. CµRTAMO    68.0

In [33]:
df_reducido['presentacion'].unique()

array(['FRASCO GOTERO 24 ML. 1.000 G., SOLUCIàN GOTAS',
       'CAJA CON 30 TABLETAS DE 50 MG.', 'BOTELLA 1 LT. VEGETAL',
       'BOTELLA 800 ML. VEGETAL', 'BOTELLA 946 ML. CANOLA',
       'BOTELLA 946 ML. SOYA', 'BOTELLA 765 ML. MAÖZ',
       'BOTELLA 825 ML. CµRTAMO',
       'BOTELLA 850 ML. VEGETAL. SABOR MANTEQUILLA', 'MANOJO',
       'CAJA CON TUBO CON 5 GR. CREMA 5%',
       'CAJA O FRASCO CON 25 COMPRIMIDOS DE 200 MG.',
       'CAJA CON 20 TABLETAS DE 5 MG.',
       'CAJA CON 5 AMPOLLETAS. SOLUCIàN 3 ML. BEBIBLE',
       'CAJA CON 10 FRASCOS CON 7 ML. 400 MG./7ML. SOLUCIàN',
       'CAJA CON 4 TABLETAS DE 37.5 MG./500 MG.',
       'CAJA CON 1+1 CµPSULA DE 150 MG.', 'CAJA CON 10 TABLETAS',
       '1 KG. GRANEL. HASS', 'CAJA CON 32 CµPSULAS DE 100 MG./300 MG.',
       'FRASCO 113 GR. A PARTIR DE LOS 6 MESES. MANZANA',
       'FRASCO 170 GR. A PARTIR DE LOS 8 MESES. MANZANA',
       'CAJA CON 20 TABLETAS DE 300 MG.',
       'CAJA CON 28 TABLETAS DE 50 MG.',
       'FRASCO CON 150 M

# Modelado

Matemáticamente, el problema se puede plantear así: Dado un carrito de compra con volumen $C$, y una lista de $n$ productos $\{p_1,p_2,...,p_n\}$ con respectivos volúmenes $\{v_1,v_2,...,v_n\}$ y costos $\{ c_1, c_2, ..., c_n \}$, se quiere escoger la combinación de enteros no negativos $\{n_1,n_2,...,n_n\}$ tal que el costo total $\sum_i n_i c_i$ sea máximo. Todo bajo la restricción $\sum_i n_i v_i \leq C$. Cada producto puede escogerse más de una vez.

Una posible solución al problema se vería como una tupla de $n$ números enteros no negativos $\{n_1,n_2,...,n_n\}$. Y en este caso la función de costo sería $\sum_i n_i c_i$ y se quiere maximizar.

Podemos tomar como valor del volumen del carrito de compras 140 litros (=140 000 ml = 140 000 cm$^3$), por tanto $C=140000$.

Ya tenemos los productos y su precio, nos falta obtener el volumen de cada producto. Esto podría obtenerse de la información contenida en la columna 'presentación', pero se necesita un poco de procesamiento para esto.

Primero, probamos un cambio de codificación de 'latin1' a 'utf-8' para corregir las letras con tildes:

In [34]:
df_reducido["presentacion"] = (
    df_reducido["presentacion"]
        .astype(str)
        .str.encode("latin1", errors="ignore")
        .str.decode("utf-8", errors="ignore")
)
df_reducido['presentacion'].unique()

/var/folders/8v/yjrt88dj0xl6q8781t51n9hm0000gp/T/ipykernel_9191/108241003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reducido["presentacion"] = (


array(['FRASCO GOTERO 24 ML. 1.000 G., SOLUCIN GOTAS',
       'CAJA CON 30 TABLETAS DE 50 MG.', 'BOTELLA 1 LT. VEGETAL',
       'BOTELLA 800 ML. VEGETAL', 'BOTELLA 946 ML. CANOLA',
       'BOTELLA 946 ML. SOYA', 'BOTELLA 765 ML. MAZ',
       'BOTELLA 825 ML. CRTAMO',
       'BOTELLA 850 ML. VEGETAL. SABOR MANTEQUILLA', 'MANOJO',
       'CAJA CON TUBO CON 5 GR. CREMA 5%',
       'CAJA O FRASCO CON 25 COMPRIMIDOS DE 200 MG.',
       'CAJA CON 20 TABLETAS DE 5 MG.',
       'CAJA CON 5 AMPOLLETAS. SOLUCIN 3 ML. BEBIBLE',
       'CAJA CON 10 FRASCOS CON 7 ML. 400 MG./7ML. SOLUCIN',
       'CAJA CON 4 TABLETAS DE 37.5 MG./500 MG.',
       'CAJA CON 1+1 CPSULA DE 150 MG.', 'CAJA CON 10 TABLETAS',
       '1 KG. GRANEL. HASS', 'CAJA CON 32 CPSULAS DE 100 MG./300 MG.',
       'FRASCO 113 GR. A PARTIR DE LOS 6 MESES. MANZANA',
       'FRASCO 170 GR. A PARTIR DE LOS 8 MESES. MANZANA',
       'CAJA CON 20 TABLETAS DE 300 MG.',
       'CAJA CON 28 TABLETAS DE 50 MG.',
       'FRASCO CON 150 ML. JARA

Ahora se 'limpian' los productos: se pasa todo a minúsculas; se eliminan acentos; se eliminan caracteres raros; y se conservan solo letras y espacios.

In [37]:
def normalizar_producto(p):
    if pd.isna(p):
        return None
    
    # pasar a minúsculas
    p = p.lower()
    
    # eliminar acentos usando unicodedata (estándar)
    p = ''.join(
        c for c in unicodedata.normalize('NFD', p)
        if unicodedata.category(c) != 'Mn'
    )
    
    # reemplazar cualquier cosa que no sea letra/número/espacio
    p = re.sub(r"[^a-z0-9 ]", " ", p)
    
    # colapsar espacios múltiples
    p = re.sub(r"\s+", " ", p).strip()
    
    return p

df_reducido['presentacion_normalizado'] = df_reducido['presentacion'].apply(normalizar_producto)
df_reducido['presentacion_normalizado'].unique()

/var/folders/8v/yjrt88dj0xl6q8781t51n9hm0000gp/T/ipykernel_9191/446052798.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reducido['presentacion_normalizado'] = df_reducido['presentacion'].apply(normalizar_producto)


array(['frasco gotero 24 ml 1 000 g solucin gotas',
       'caja con 30 tabletas de 50 mg', 'botella 1 lt vegetal',
       'botella 800 ml vegetal', 'botella 946 ml canola',
       'botella 946 ml soya', 'botella 765 ml maz',
       'botella 825 ml crtamo',
       'botella 850 ml vegetal sabor mantequilla', 'manojo',
       'caja con tubo con 5 gr crema 5',
       'caja o frasco con 25 comprimidos de 200 mg',
       'caja con 20 tabletas de 5 mg',
       'caja con 5 ampolletas solucin 3 ml bebible',
       'caja con 10 frascos con 7 ml 400 mg 7ml solucin',
       'caja con 4 tabletas de 37 5 mg 500 mg',
       'caja con 1 1 cpsula de 150 mg', 'caja con 10 tabletas',
       '1 kg granel hass', 'caja con 32 cpsulas de 100 mg 300 mg',
       'frasco 113 gr a partir de los 6 meses manzana',
       'frasco 170 gr a partir de los 8 meses manzana',
       'caja con 20 tabletas de 300 mg', 'caja con 28 tabletas de 50 mg',
       'frasco con 150 ml jarabe adulto',
       'frasco con 150 ml jara

Ahora, se definirá una función que toma la columna `presentacion_normalizado` y devuelve un volumen estimado en cm³.

La función sigue este orden lógico:

Si encuentra volumen explícito en (ml, L), lo usa.

Si encuentra masa en (g, kg), convierte usando densidades por categorías de productos.

Si detecta medicamentos (tabletas, cápsulas, comprimidos, ampolletas), usa heurísticas para asignar un volumen.

Si es granel, usa la densidad promedio del producto fresco.

Si nada aplica, entonces asigna un volumen estándar según palabras clave (botella, lata, bolsa, caja).

In [41]:
# -------------------------------------------
# 1. Densidades aproximadas por categoría
# -------------------------------------------
DENSIDADES = {
    "arroz": 0.85,
    "frijol": 0.80,
    "azucar": 0.90,
    "harina": 0.60,
    "cereal": 0.25,
    "atun": 1.00,
    "aceite": 0.92,
    "pollo": 1.05,
    "carne": 1.05,
    "verdura": 0.60,
    "lata": 1.00
}

# Volúmenes estándar por tipo de envase cuando no se encuentra información numérica
VOLUMEN_STANDAR = {
    "botella": 1000,
    "frasco": 250,
    "lata": 355,
    "bolsa": 500,
    "caja": 300,
    "paquete": 300,
    "manojo": 300,
}

# -------------------------------------------
# Función auxiliar: extraer número antes de unidad
# -------------------------------------------
def extraer_numero(texto, patron):
    m = re.search(patron, texto)
    if m:
        return float(m.group(1))
    return None

# -------------------------------------------
# Función principal
# -------------------------------------------
def estimar_volumen(texto):
    texto = texto.lower()

    # ----------------------------
    # 1. Detectar ml y litros
    # ----------------------------
    ml = extraer_numero(texto, r"(\d+)\s*ml")
    if ml:
        return ml  # 1 ml = 1 cm3

    litros = extraer_numero(texto, r"(\d+(\.\d+)?)\s*l")
    if litros:
        return litros * 1000

    # ----------------------------
    # 2. Detectar masa (g o kg)
    # ----------------------------
    gramos = extraer_numero(texto, r"(\d+)\s*g(?!ranel)")  # evitar capturar "granel 1 kg"
    if not gramos:
        kg = extraer_numero(texto, r"(\d+)\s*kg")
        if kg:
            gramos = kg * 1000

    if gramos:
        # Buscar palabras clave para densidad
        for palabra, densidad in DENSIDADES.items():
            if palabra in texto:
                return gramos / densidad
        
        # Si no se identifica categoría, asumir densidad agua
        return gramos / 1.0

    # ----------------------------
    # 3. Medicamentos (tabletas, cápsulas, etc.)
    # ----------------------------
    if any(x in texto for x in ["tableta", "tabletas", "cápsula", "capsula", "cpsula", "comprimido"]):
        # Buscar número de unidades
        unidades = extraer_numero(texto, r"(\d+)\s*(tableta|cápsula|capsula|cpsula|comprimido)")
        if unidades:
            # Heurística: 2–3 cm3 por tableta en caja compacta
            return unidades * 2.5
        # Si no se detecta número, asignar tamaño de caja pequeño
        return 30

    # Ampolletas y jeringas
    if "ampolleta" in texto or "jeringa" in texto:
        ml = extraer_numero(texto, r"(\d+)\s*ml")
        if ml:
            # Añadir volumen del empaque
            return ml + 20  
        return 40  # estimación genérica

    # ----------------------------
    # 4. Granel ("1 kg granel")
    # ----------------------------
    if "granel" in texto:
        gramos = extraer_numero(texto, r"(\d+)\s*kg")
        if gramos:
            gramos *= 1000
            # Densidad promedio de alimentos frescos
            return gramos / 0.95
        return 800  # estimación genérica

    # ----------------------------
    # 5. Volumen estándar por envase
    # ----------------------------
    for palabra, vol in VOLUMEN_STANDAR.items():
        if palabra in texto:
            return vol

    # ----------------------------
    # 6. Si nada funcionó, devolver NaN
    # ----------------------------
    return np.nan


Y ahora generaremos una nueva columna en el DataFrame df_reducido que contenga el volumen estimado por la función:

In [44]:
df_reducido["volumen_estimado_cm3"] = df_reducido["presentacion_normalizado"].apply(estimar_volumen)
df_reducido.head(20)

/var/folders/8v/yjrt88dj0xl6q8781t51n9hm0000gp/T/ipykernel_9191/1831414310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reducido["volumen_estimado_cm3"] = df_reducido["presentacion_normalizado"].apply(estimar_volumen)


producto                                       presentacion  \
488919       A.S.COR       FRASCO GOTERO 24 ML. 1.000 G., SOLUCIN GOTAS   
488920      ACARBOSA                     CAJA CON 30 TABLETAS DE 50 MG.   
488921        ACEITE                              BOTELLA 1 LT. VEGETAL   
488922        ACEITE                            BOTELLA 800 ML. VEGETAL   
488923        ACEITE                             BOTELLA 946 ML. CANOLA   
488924        ACEITE                             BOTELLA 946 ML. CANOLA   
488925        ACEITE                               BOTELLA 946 ML. SOYA   
488926        ACEITE                                BOTELLA 765 ML. MAZ   
488927        ACEITE                               BOTELLA 946 ML. SOYA   
488928        ACEITE                             BOTELLA 825 ML. CRTAMO   
488929        ACEITE         BOTELLA 850 ML. VEGETAL. SABOR MANTEQUILLA   
488930        ACELGA                                             MANOJO   
488931     ACICLOVIR                   CAJA CON TUBO CON 5 GR. CREMA 5%   
488932     ACICLOVIR        CAJA O FRASCO CON 25 COMPRIMIDOS DE 200 MG.   
488933  ACIDO FàLICO                      CAJA CON 20 TABLETAS DE 5 MG.   
488934   ADEROGYL 15       CAJA CON 5 AMPOLLETAS. SOLUCIN 3 ML. BEBIBLE   
488935        ADIMOD  CAJA CON 10 FRASCOS CON 7 ML. 400 MG./7ML. SOL...   
488936        AFUMIX            CAJA CON 4 TABLETAS DE 37.5 MG./500 MG.   
488937       AFUNGIL                     CAJA CON 1+1 CPSULA DE 150 MG.   
488938       AGRIFEN                               CAJA CON 10 TABLETAS   

        precio                         presentacion_normalizado  \
488919   560.0        frasco gotero 24 ml 1 000 g solucin gotas   
488920    83.0                    caja con 30 tabletas de 50 mg   
488921    38.0                             botella 1 lt vegetal   
488922    30.0                           botella 800 ml vegetal   
488923    37.0                            botella 946 ml canola   
488924    37.0                            botella 946 ml canola   
488925    38.0                              botella 946 ml soya   
488926    62.0                               botella 765 ml maz   
488927    45.0                              botella 946 ml soya   
488928    68.0                            botella 825 ml crtamo   
488929    42.0         botella 850 ml vegetal sabor mantequilla   
488930    12.9                                           manojo   
488931    54.0                   caja con tubo con 5 gr crema 5   
488932   118.0       caja o frasco con 25 comprimidos de 200 mg   
488933    21.0                     caja con 20 tabletas de 5 mg   
488934   165.0       caja con 5 ampolletas solucin 3 ml bebible   
488935   772.0  caja con 10 frascos con 7 ml 400 mg 7ml solucin   
488936   800.0            caja con 4 tabletas de 37 5 mg 500 mg   
488937   559.0                    caja con 1 1 cpsula de 150 mg   
488938    35.0                             caja con 10 tabletas   

        volumen_estimado_cm3  
488919                  24.0  
488920                  75.0  
488921                1000.0  
488922                 800.0  
488923                 946.0  
488924                 946.0  
488925                 946.0  
488926                 765.0  
488927                 946.0  
488928                 825.0  
488929                 850.0  
488930                 300.0  
488931                   5.0  
488932                  62.5  
488933                  50.0  
488934                   3.0  
488935                   7.0  
488936                  10.0  
488937                   2.5  
488938                  25.0

Vemos que la aproximación es suficientemente buena para la mayoría de los casos, aunque no para todos.
Ahora teniendo ya los nombres de los productos, su precio y su volumen aproximado, podemos centrarnos en resolver el problema de optimización, pero antes, reducimos el dataframe a solo las columnas que nos interesan: el nombre, el precio y el volumen:

In [53]:
df_final = df_reducido.reset_index()[['producto','precio','volumen_estimado_cm3']]
df_final.head(-5)

producto  precio  volumen_estimado_cm3
0        A.S.COR   560.0                  24.0
1       ACARBOSA    83.0                  75.0
2         ACEITE    38.0                1000.0
3         ACEITE    30.0                 800.0
4         ACEITE    37.0                 946.0
...          ...     ...                   ...
1023  VITACILINA    52.0                  28.0
1024      WHISKY   435.0                 700.0
1025      WHISKY   275.0                 700.0
1026        XL-3    49.0                  25.0
1027        XL-3   125.0                  60.0

[1028 rows x 3 columns]

# Búsqueda exhaustiva

En principio, el tamaño del espacio de soluciones es infinito, porque los $\{n_1,n_2,...,n_n\}$ pueden tomar como valor cualquier entero no negativo.

Pero con la restricción de capacidad y suponiendo que todos los volúmenes son positivos, pasa algo muy importante:

Para cada producto $i$:
$v_i n_i \leq C$ ⇒ $n_i \le \left\lfloor \frac{C}{v_i} \right\rfloor$

Es decir, cada $n_i$ tiene un máximo posible.
Por ejemplo, si 𝐶=50,000 cm³ y $𝑣_𝑖$=500 cm³:

$n_i \le \left\lfloor \frac{50,000}{500} \right\rfloor = 100$

Entonces:

Cada variable $n_i$ solo puede tomar un número finito de valores:
0,1,2,...,$\left\lfloor \frac{C}{v_i} \right\rfloor$

Por lo tanto, el número de combinaciones posibles es finito.

Un límite superior (no exacto, pero sencillo) para el tamaño del espacio de búsqueda es:

$N_{\text{soluciones}} \le \prod_{i=1}^{n} \left( \left\lfloor \frac{C}{v_i} \right\rfloor + 1 \right)$


Así que:

Matemáticamente, el espacio de soluciones factibles es finito, mientras todos los volúmenes sean > 0 y la capacidad del carrito sea finita. Pero aunque sea finito, puede ser astronómicamente grande.

Calculamos el valor de $n$, el tamaño del grupo de productos.

In [59]:
#n = df_final.shape[0]
n = len(df_final)
print("n =",n)

n = 1033


Se puede estimar una cota superior:

$N_{\text{soluciones}} \le \left( \left\lfloor \frac{C}{v_{min}} \right\rfloor + 1 \right)^n$

donde $𝑣_{min} = min_i ⁡𝑣_𝑖$.

In [64]:
v_min = min(df_final['volumen_estimado_cm3'])
print("v_min =", v_min, " cm^3")

v_min = 1.0  cm^3


In [63]:
C=140000
raiz_n_cota_sup = ( C//v_min + 1 )
raiz_n_cota_sup

140001.0

Por tanto, una cota superior aproximada para el número de soluciones sería $140,001^{1033}$ o casi igual a $14^{4132}$ (un número enorme)

Supongamos por simplicidad que mi computadora puede evaluar $10^6$ soluciones por segundo (que es optimista pero sirve de ejemplo).

Entonces, el tiempo para calcular todo el espacio de soluciones sería:

$𝑇_{segundos}≈𝑁_{soluciones}/10^6$ = $14^{4132} / 10^6$ segundos, que sigue siendo un número grandísimo (La edad del universo es de aproximadamente $4.35x10^{17}$ segundos). Esto se debe pricipalmente a que el conjunto de productos únicos del súper es muy grande (1033 productos).

# Muestreo aleatorio
Un muestreo aleatorio de soluciones significa generar vectores factibles al azar, sin revisar el espacio completo.

Esto se utiliza para: estimar el espacio de búsqueda, tener “puntos de referencia” antes de la optimización o incluso para obtener soluciones base para un algoritmo metaheurístico.

El problema es que muestrear directamente los $𝑛_𝑖$ rara vez produce soluciones factibles. El truco es ir agregando productos mientras haya capacidad, no al revés.

Un algoritmo simple para generar una solución aleatoria factible sería (tipo “llenado aleatorio con rechazo”):

1. Empiezas con la capacidad restante:

$𝑅=𝐶$

2. Mientras $𝑅>0$:

-Elige un producto al azar.

-Si su volumen cabe en el carrito ($𝑣_𝑖≤𝑅$), agrégalo:

$𝑛_𝑖 ← 𝑛_𝑖 + 1$

$𝑅 ← 𝑅−𝑣_𝑖$

-Si no cabe, selecciona otro.

3. Detén cuando ningún producto pueda entrar.

Ya implementado se vería así:

In [70]:
import random
import numpy as np

def muestrear_solucion(df, C):
    vol = df["volumen_estimado_cm3"].values
    n = len(vol)
    solucion = np.zeros(n, dtype=int)
    
    capacidad_restante = C
    
    # Lista de índices de productos que pueden caber
    indices_validos = [i for i in range(n) if vol[i] > 0]
    
    while True:
        # Filtrar solo productos que aún caben
        caben = [i for i in indices_validos if vol[i] <= capacidad_restante]
        if not caben:
            break  # no cabe nada más
        
        # Elegir uno al azar
        i = random.choice(caben)
        
        # Agregarlo
        solucion[i] += 1
        capacidad_restante -= vol[i]
    
    return solucion

Usándolo, podemos ver:

In [71]:
sol_muestreo = muestrear_solucion(df_reducido, C)
sol_muestreo

array([0, 0, 1, ..., 1, 1, 0])

Pero esa es la representación abstracta de la solución, en términos de los artículos/productos se vería así:

In [79]:
df_final.assign(n=sol_muestreo).query("n > 0")

producto  precio  volumen_estimado_cm3  n
2                            ACEITE    38.0                1000.0  1
12                        ACICLOVIR    54.0                   5.0  1
17                           AFUMIX   800.0                  10.0  2
22    ALIMENTO PREPARADO PARA NI¥OS    17.0                 113.0  1
23    ALIMENTO PREPARADO PARA NI¥OS    22.0                 170.0  1
...                             ...     ...                   ... ..
1017                    VENLAFAXINA   305.0                  50.0  1
1024                         WHISKY   435.0                 700.0  1
1029                    YASMIN 24/4   718.0                  70.0  1
1030                      ZANAHORIA    14.9                1000.0  1
1031                      ZANAHORIA    14.9                1000.0  1

[265 rows x 4 columns]

# Método de optimización

Como método de optimización se escogió el método de Temple Simulado (Simulated Annealing) porque es fácil de implementar y es muy bueno para problemas discretos, como el que estamos trabajando (su desventaja es que hay que "jugar" con los valores de temperatura, los cuales no están establecidos de manera objetiva):

In [69]:
import math

def simulated_annealing_knapsack(values, weights, W,
                                 T_init=1000,     # Temperatura inicial
                                 T_min=1e-3,      # Temperatura mínima
                                 alpha=0.99,      # Factor de enfriamiento
                                 max_iter=5000):  # Número máximo de iteraciones

    n = len(values)

    # -------------------------
    # Funciones auxiliares
    # -------------------------

    def peso(sol):
        return sum(sol[i] * weights[i] for i in range(n))

    def valor(sol):
        return sum(sol[i] * values[i] for i in range(n))

    def generar_vecino(sol):
        """
        Genera una solución vecina modificando ligeramente la actual:
        - Incrementa o decrementa la cantidad de un ítem.
        - Repara la solución si excede la capacidad.
        """
        nueva = sol[:]

        i = random.randint(0, n - 1)

        # Decide si sumar o restar unidades
        if random.random() < 0.5:
            nueva[i] += 1
        else:
            if nueva[i] > 0:
                nueva[i] -= 1

        # Reparar si excede la capacidad
        while peso(nueva) > W:
            j = random.randint(0, n - 1)
            if nueva[j] > 0:
                nueva[j] -= 1

        return nueva

    # -------------------------
    # Inicializar solución
    # -------------------------
    solucion_actual = [0] * n
    mejor_solucion = solucion_actual[:]

    T = T_init

    # -------------------------
    # Bucle principal del temple simulado
    # -------------------------
    for _ in range(max_iter):

        vecino = generar_vecino(solucion_actual)

        delta = valor(vecino) - valor(solucion_actual)

        # Regla de aceptación
        if delta > 0:
            solucion_actual = vecino
        else:
            prob = math.exp(delta / T)
            if random.random() < prob:
                solucion_actual = vecino

        # Actualizar mejor solución encontrada
        if valor(solucion_actual) > valor(mejor_solucion):
            mejor_solucion = solucion_actual[:]

        # Enfriamiento
        T *= alpha
        if T < T_min:
            break

    # -------------------------
    # Retorno final
    # -------------------------
    return mejor_solucion, valor(mejor_solucion), peso(mejor_solucion)

In [78]:
precios = df_final['precio']
volumenes = df_final['volumen_estimado_cm3']
#para rellenar datos vacíos y así evitar que el volumen total sea 'nan':
volumenes = volumenes.fillna(volumenes.mean())
C = 140000 #capacidad del carrito

sol, val, vt = simulated_annealing_knapsack(precios, volumenes, C)
print("Mejor solución encontrada:", sol)
print("Valor total:", val)
print("Volúmen total:", vt)

Mejor solución encontrada: [1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 3, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 3, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 2, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 

In [80]:
df_final.assign(n=sol).query("n > 0")

producto  precio  volumen_estimado_cm3  n
0            A.S.COR   560.0                  24.0  1
7             ACEITE    62.0                 765.0  2
15       ADEROGYL 15   165.0                   3.0  3
16            ADIMOD   772.0                   7.0  2
18           AFUNGIL   559.0                   2.5  1
...              ...     ...                   ... ..
1015  VENDA ELµSTICA    62.0                 300.0  1
1023      VITACILINA    52.0                  28.0  5
1025          WHISKY   275.0                 700.0  1
1026            XL-3    49.0                  25.0  1
1028          YASMIN   678.0                  52.5  1

[280 rows x 4 columns]

Vemos que la solución óptima consiste en poner en el carrito: 2 aceites de 765 ml, 3 Aderogyl de 3 ml, 2 Adimod, 1 Afungil, 5 Vitacilinas de 29 ml, 1 botella de Whisky de 700 ml, entre otros artículos.
Con esto, mi padre podría tener una idea muy clara de qué poner en el carrito si alguna vez le tocase hacer la dinámica de nuevo. Si tan solo estuviera aquí...